# 전이학습

## 텐서플로우 applications를 이용한 전이학습

[https://www.tensorflow.org/api_docs/python/tf/keras/applications](https://www.tensorflow.org/api_docs/python/tf/keras/applications)

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import layers, Model

### 데이터

In [ ]:
from tensorflow.keras import datasets, utils

(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
# 데이터 전처리 x: minmax min0 max255 -> x/255
x_train, x_test = x_train/255, x_test/255
# 데이터 전처리 y: categorical
y_train = utils.to_categorical(y_train, num_classes=10)
y_test = utils.to_categorical(y_test, num_classes=10)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

170508288/170498071 [==============================] - 3s 0us/step


((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

### 모델 구현

* 모델 불러오기

In [ ]:
base_model = applications.vgg16.VGG16(weights='imagenet', 
                                      include_top=False, # 마지막 레이어
                                      input_shape=[32,32,3])
base_model.summary()

58900480/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
______________________________________________________________

* 모델 조합하기

In [ ]:
from tensorflow.keras import layers, models

x = base_model.output
y = layers.Flatten(name='Add_flatten')(x)
#y = layers.Dense(128, activation='relu')(y)
y = layers.Dense(10, activation='softmax', name='Add_dense')(y)
model = models.Model(inputs=base_model.input, outputs = y)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

* 컴파일

In [ ]:
## base model의 weight들은 학습을 안하고싶다!
for i in range(19):
  print(i, model.layers[i])
  model.layers[i].trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['acc'])

### 학습

In [ ]:
history = model.fit(x_train, y_train, batch_size=16, epochs=1)

3125/3125 [==============================] - 560s 179ms/step - loss: 1.1107 - acc: 0.6167


### 평가

In [ ]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 107s 341ms/step - loss: 1.2041 - acc: 0.5825


[1.2041215896606445, 0.5824999809265137]

* 시각화

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['loss'])
plt.show()